In [ ]:
import pandas as pd
import sklearn
import os
import numpy as np
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,r2_score, f1_score, accuracy_score, precision_score, recall_score,confusion_matrix,make_scorer
from sklearn import linear_model, metrics, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn import neighbors, tree, ensemble, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import metrics, preprocessing, neighbors, cluster
from sklearn import cluster
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
%matplotlib inline

from scipy.special import expit
from scipy.special import logit

In [ ]:
stats_df = pd.read_csv('C:\prject\player_stats_cleaned.csv',index_col = 0)
schedule_df = pd.read_csv(r'C:\prject\all_teams_schedule_2010_2020_cleaned.csv',index_col = 0)
missed_games_df = pd.read_csv('C:\prject\missedgames_cleaned.csv',index_col = 0)
important_missed_games= pd.read_csv(r'C:\prject\important_missedgames_cleaned.csv',index_col = 0)
df_game_number= pd.read_csv(r'C:\prject\df_game_number.csv',index_col = 0)
df_before=pd.read_csv(r'C:\prject\before_important_missedgames_cleaned.csv')

In [ ]:
df_befores=df_before.copy()
df_befores.drop_duplicates(subset=['Player','Year'],keep='first',inplace=True)
df_befores.drop(columns=['Unnamed: 0'],inplace=True)
df_befores.reset_index(drop=True, inplace=True)
df_befores['Notes'] = df_befores['Notes'].fillna(0)
df_befores['Notes'] = df_befores['Notes'].str.extract('(\d*)', expand=False)
mapping = {'':1,None:0}
df_befores.replace({'Notes': mapping},inplace=True)
df = pd.DataFrame({"Notes": df_befores['Notes'], "G": df_befores['G']})

In [ ]:
lrm=linear_model.LogisticRegression()
lrm.fit(df[["G"]],df["Notes"])
df.plot.scatter(x='G',y='Notes',s=30)
x = np.linspace(df['G'].min(),df['G'].max(),1000)
y_pred_p=lrm.predict_proba(np.reshape(x, (-1, 1)))
y_pred_p1=[y[1] for y in y_pred_p]
plt.plot(x,y_pred_p1,color='Black')
plt.plot(x,[0.5]*len(x),color="red")
plt.show()

In [ ]:
prob_passing = 0.5
print(logit(prob_passing) - lrm.intercept_[0]/lrm.coef_[0][0])
G=30
print(expit(lrm.intercept_[0]+lrm.coef_[0][0]*G))
y_pred = lrm.predict(df[["G"]])
y_actual=df["Notes"]
metrics.confusion_matrix(y_actual, y_pred)
print("accuracy is:",metrics.accuracy_score(y_actual, y_pred))
print("precision is:",metrics.precision_score(y_actual, y_pred))
print("recall is:",metrics.recall_score(y_actual, y_pred))
print("f1 is:",metrics.f1_score(y_actual, y_pred))

In [ ]:
df_pre=df_befores.copy()
df_pre.drop(["Player","Pos","Year","Team"],axis=1,inplace=True)
used_features = ["G", "MPPG","BLK","STL","DRB","ORB","FTA","2PA","3PA","FGA","Age","GS"]
injury_pred = df_pre[used_features].values
injury_labels = df_pre["Notes"].values
XTrain, XTest, yTrain, yTest = train_test_split(injury_pred, injury_labels, random_state=1, test_size=0.5)
decisionTree = tree.DecisionTreeClassifier()
decisionTree = decisionTree.fit(XTrain, yTrain)
y_pred_train = decisionTree.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))
y_pred = decisionTree.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

In [ ]:
forest =  RandomForestClassifier(bootstrap=True, n_estimators=300, random_state=0)
trained_forest = forest.fit(XTrain, yTrain) 
y_pred_train = trained_forest.predict(XTrain)
print('Accuracy on training data= ', metrics.accuracy_score(y_true = yTrain, y_pred = y_pred_train))
y_pred = trained_forest.predict(XTest)
print('Accuracy on test data= ', metrics.accuracy_score(y_true = yTest, y_pred = y_pred))

In [ ]:
featuresDF=pd.DataFrame({"features":used_features,"weights":trained_forest.feature_importances_})
featuresDF.sort_values("weights",ascending=False)

In [ ]:
def notes_filter(c):
    note = c['Notes']
    lower_case_note =note.lower()
    if any(x in lower_case_note for x in ['return', 'returned','activate', 'activated']):
        return 'returned to lineup', '0'
    elif lower_case_note == 'placed on il' or lower_case_note == 'placed on il (p)':
        return 'roster move', '0'
    elif 'suspension' in lower_case_note:
        return 'suspension', '0'
    elif any(x in lower_case_note for x in ['family','personal','birth', 'death']):
        return 'personal reasons', '0'
    elif 'rest' in lower_case_note:
        return 'rest', '0'
    elif any(x in lower_case_note for x in ['virus','headache','flu', 'sick', 'illness','infection','pneumonia', 'gastro','appende','nausea', 'pox', 'dizziness', 'poisoning','bronchitis']):
        return 'sick', '0'
    elif 'foot' in lower_case_note:
        return 'foot', '1'
    elif 'toe' in lower_case_note:
        return 'toe', '1'
    elif 'heel' in lower_case_note:
        return 'heel', '1'
    elif 'ankle' in lower_case_note:
        return 'ankle', '2'             
    elif 'achilles'in lower_case_note:
        return 'achilles', '2'  
    elif 'calf' in lower_case_note:
        return 'calf', '2'
    elif 'shin' in lower_case_note:
        return 'shin', '2'
    elif 'tibia' in lower_case_note:
        return 'tibia', '2'
    elif 'fibula' in lower_case_note:
        return 'fibula','2'
    elif 'acl' in lower_case_note:
        return 'ACL', '3'
    elif 'mcl' in lower_case_note:
        return 'MCL', '3'
    elif any(x in lower_case_note for x in ['knee','patella','meniscus']):
        return 'knee', '3'
    elif any(x in lower_case_note for x in ['quad','quadriceps','thigh']):
        return 'quad', '4'
    elif 'hamstring' in lower_case_note:
        return 'hamstring', '4'
    elif 'groin' in lower_case_note:
        return 'groin', '4'
    elif any(x in lower_case_note for x in ['hip','adductor']):
        return 'hip', '4'
    elif 'femur' in lower_case_note:
        return 'femur', '4'
    elif 'leg' in lower_case_note:
        return 'leg', '4'
    elif any(x in lower_case_note for x in ['chest', 'pectoral']):
        return 'chest', '5'
    elif any(x in lower_case_note for x in ['shoulder','rotator cuff']):
        return 'shoulder', '5'
    elif 'back' in lower_case_note:
        return 'back', '5'
    elif 'collarbone' in lower_case_note:
        return 'collarbone', '5'
    elif 'rib' in lower_case_note:
        return 'ribs', '5'
    elif any(x in lower_case_note for x in ['abdom','abductor','oblique']):
        return 'abdominal', '5'
    elif 'neck' in lower_case_note:
        return 'neck', '6'
    elif any(x in lower_case_note for x in ['head', 'concussion']):
        return 'head', '6'
    elif 'eye' in lower_case_note:
        return 'eye', '6'
    elif 'nose' in lower_case_note:
        return 'nose', '6'
    elif 'hand' in lower_case_note:
        return 'hand', '6'
    elif any(x in lower_case_note for x in ['finger', 'thumb']):
        return 'finger', '6'
    elif 'arm' in lower_case_note:
        return 'arm', '7'
    elif 'elbow' in lower_case_note:
        return 'elbow', '7'
    elif 'bicep' in lower_case_note:
        return 'bicep', '7'
    elif 'tricep' in lower_case_note:
        return 'tricep', '7'
    elif 'wrist' in lower_case_note:
        return 'wrist', '7'     
    else:
        return 'other', '0'

In [ ]:
df_knn=important_missed_games.copy()
df_knn['area'] = ''
df_knn['category'] = ''
df_knn[['area', 'category']]= df_knn.apply(notes_filter, axis = 1, result_type="expand")
df_knn.drop(['area','Notes','Player','Team','Pos'],axis=1,inplace=True)

In [ ]:
y=df_knn["category"]
X=df_knn
X.drop(["category"],axis=1,inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, test_size=0.7)
scaler = StandardScaler() 
scaler.fit(X) 
scaled_features = scaler.transform(X)

In [ ]:
error_rate = [] 
for i in range(1, 40): 
    knn = KNeighborsClassifier(n_neighbors = i) 
    knn.fit(X_train, y_train) 
    pred_i = knn.predict(X_test) 
    error_rate.append(np.mean(pred_i != y_test)) 
plt.figure(figsize =(10, 6)) 
plt.plot(range(1, 40), error_rate, color ='blue', 
                linestyle ='dashed', marker ='o', 
         markerfacecolor ='red', markersize = 10) 
plt.title('Error Rate vs. K Value') 
plt.xlabel('K') 
plt.ylabel('Error Rate') 

In [ ]:
knn=KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train, y_train) 
pred = knn.predict(X_test) 
print(confusion_matrix(y_test, pred)) 
print(classification_report(y_test, pred)) 